## Eccentric Residual Search

This notebook tests on a simulated dataset containing an eccentric gw signal. Based on work done by Sarah Vigeland, Ph.D. from `cw_search_sample.ipynb`

Updated: 05/08/2020

In [1]:
from __future__ import division
import numpy as np
import glob
import os
import pickle
import json
import matplotlib.pyplot as plt
import corner
import sys

from enterprise.signals import parameter
from enterprise.pulsar import Pulsar
from enterprise.signals import selections
from enterprise.signals import signal_base
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise.signals import utils
from enterprise_extensions.deterministic import CWSignal

import libstempo as T
import libstempo.plot as LP, libstempo.toasim as LT
import ecc_res

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

import enterprise_cw_funcs as fns
#np.set_printoptions(threshold=sys.maxsize)

%load_ext autoreload
%autoreload 2

Cannot import PINT? Meh...


All the noise files must be loaded in using the following function `get_noise_from_pal2` because these noise files are all text files. If using other file types then this is function is not necessary.

In [2]:
def get_noise_from_pal2(noisefile):
    psrname = noisefile.split('/')[-1].split('_noise.txt')[0]
    fin = open(noisefile, 'r')
    lines = fin.readlines()
    params = {}
    for line in lines:
        ln = line.split()
        if 'efac' in line:
            par = 'efac'
            flag = ln[0].split('efac-')[-1]
        elif 'equad' in line:
            par = 'log10_equad'
            flag = ln[0].split('equad-')[-1]
        elif 'jitter_q' in line:
            par = 'log10_ecorr'
            flag = ln[0].split('jitter_q-')[-1]
        elif 'RN-Amplitude' in line:
            par = 'red_noise_log10_A'
            flag = ''
        elif 'RN-spectral-index' in line:
            par = 'red_noise_gamma'
            flag = ''
        else:
            break
        if flag:
            name = [psrname, flag, par]
        else:
            name = [psrname, par]
        pname = '_'.join(name)
        params.update({pname: float(ln[1])})
    return params

In [3]:
#Simulated dataset directory path
datadir = '/home/bcheeseboro/nanograv_proj/enterprise_proj/eccentric_residual/ecc_sim_data/testing_round'

In [4]:
#load par and tim files for each of the pulsars
parfiles = sorted(glob.glob(datadir+'/*.par'))
timfiles = sorted(glob.glob(datadir+'/*.tim'))

In [5]:
#Create the pulsar list
psr_list = [x.split('/')[-1].split('_')[0] for x in parfiles]

In [6]:
#Load in pulsars as a list
psrs = []
for psr in psr_list:
    for p, t in zip(parfiles, timfiles):
        if psr in p:
            print('Loading pulsar from parfile {0}'.format(p))
            psrs.append(Pulsar(p, t))

Loading pulsar from parfile /home/bcheeseboro/nanograv_proj/enterprise_proj/eccentric_residual/ecc_sim_data/testing_round/B1855+09_simulate.par
Loading pulsar from parfile /home/bcheeseboro/nanograv_proj/enterprise_proj/eccentric_residual/ecc_sim_data/testing_round/B1937+21_simulate.par
Loading pulsar from parfile /home/bcheeseboro/nanograv_proj/enterprise_proj/eccentric_residual/ecc_sim_data/testing_round/B1953+29_simulate.par
Loading pulsar from parfile /home/bcheeseboro/nanograv_proj/enterprise_proj/eccentric_residual/ecc_sim_data/testing_round/J0023+0923_simulate.par


Loading pulsar from parfile /home/bcheeseboro/nanograv_proj/enterprise_proj/eccentric_residual/ecc_sim_data/testing_round/J0030+0451_simulate.par
Loading pulsar from parfile /home/bcheeseboro/nanograv_proj/enterprise_proj/eccentric_residual/ecc_sim_data/testing_round/J0340+4130_simulate.par


Loading pulsar from parfile /home/bcheeseboro/nanograv_proj/enterprise_proj/eccentric_residual/ecc_sim_data/testing_round/J0613-0200_simulate.par
Loading pulsar from parfile /home/bcheeseboro/nanograv_proj/enterprise_proj/eccentric_residual/ecc_sim_data/testing_round/J0636+5128_simulate.par


Loading pulsar from parfile /home/bcheeseboro/nanograv_proj/enterprise_proj/eccentric_residual/ecc_sim_data/testing_round/J0645+5158_simulate.par


Loading pulsar from parfile /home/bcheeseboro/nanograv_proj/enterprise_proj/eccentric_residual/ecc_sim_data/testing_round/J0740+6620_simulate.par


In [7]:
# white noise parameters
efac = parameter.Constant()
equad = parameter.Constant()
ecorr = parameter.Constant()

# there will be separate white noise parameters for each observing backend
# since NANOGrav began taking data, there have been two generations of backends
# (ASP and PUPPI at Arecibo, GASP and GUPPI at Green Bank)
selection = selections.Selection(selections.by_backend)

# white noise signals
ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection)

In [9]:
#Eccentric gw parameters
pdist = parameter.Normal(0,1) #puslar distance
#gw parameter
gwphi = parameter.Constant(3.1)('gwphi')
gwtheta = parameter.Constant(-np.pi/3.5)('gwtheta')

#orbital parameters
q = parameter.Uniform(0,1)('q')
log10_mc = parameter.LinearExp(7,10)('log10_mc')
F0 = parameter.Constant(1/(2*365*86400))('F0')
e0 = parameter.Uniform(0.0,0.9)('e0')
l0 = parameter.Constant(0)('l0')
gamma0 = parameter.Constant(0)('gamma0')
inc = parameter.Constant(np.pi/3)('inc')
psi = parameter.Constant(0)('psi')

In [10]:
#Eccentric signal construction
#To create a signal to be used by enterprise you must first create a residual 
#and use CWSignal to convert the residual as part of the enterprise Signal class
ewf = ecc_res.add_ecc_cgw(gwphi=gwphi, gwtheta=gwtheta,
				mc=log10_mc, q=q, F0=F0, e0=e0, l0=l0, gamma0=gamma0, inc=inc, psi=psi, pdist=pdist, gwdist=1.0, l_P=None, 
				gamma_P=None, tref=0, psrterm=True, evol=True,
				waveform_cal='Num', res ='Both')
ew = CWSignal(ewf, ecc=True, psrTerm=True)


In [11]:
# linearized timing model
tm = gp_signals.TimingModel(use_svd=False)
# full signal (no red noise added at this time)
s = ef + eq + ec + tm + ew

In [13]:
# load the white noise parameters from the noisefiles
noisedir = '/home/bcheeseboro/nanograv_proj/enterprise_proj/pta_tutorial/noisefiles_new/'
noisefiles = sorted(glob.glob(noisedir+'*_noise.txt'))
noisefiles = [x for x in noisefiles if x.split('/')[-1].split('_')[0] in psr_list]

setpars = {}
for nfile in noisefiles:
    setpars.update(get_noise_from_pal2(nfile))
setpars

{'B1855+09_430_PUPPI_efac': 1.11896,
 'B1855+09_L-wide_PUPPI_efac': 1.38104,
 'B1855+09_430_ASP_efac': 1.16587,
 'B1855+09_L-wide_ASP_efac': 1.08538,
 'B1855+09_430_ASP_log10_ecorr': -8.47348,
 'B1855+09_430_PUPPI_log10_ecorr': -6.31096,
 'B1855+09_L-wide_ASP_log10_ecorr': -6.09208,
 'B1855+09_L-wide_PUPPI_log10_ecorr': -6.401,
 'B1855+09_430_PUPPI_log10_equad': -6.17415,
 'B1855+09_L-wide_PUPPI_log10_equad': -6.53715,
 'B1855+09_430_ASP_log10_equad': -7.93502,
 'B1855+09_L-wide_ASP_log10_equad': -6.51038,
 'B1855+09_red_noise_log10_A': -13.8022,
 'B1855+09_red_noise_gamma': 3.63368,
 'B1937+21_S-wide_ASP_efac': 1.38306,
 'B1937+21_L-wide_PUPPI_efac': 2.51222,
 'B1937+21_L-wide_ASP_efac': 2.08151,
 'B1937+21_Rcvr_800_GUPPI_efac': 4.56347,
 'B1937+21_S-wide_PUPPI_efac': 4.42791,
 'B1937+21_Rcvr1_2_GASP_efac': 1.2136,
 'B1937+21_Rcvr_800_GASP_efac': 2.28379,
 'B1937+21_Rcvr1_2_GUPPI_efac': 1.51615,
 'B1937+21_L-wide_ASP_log10_ecorr': -6.73728,
 'B1937+21_L-wide_PUPPI_log10_ecorr': -6.894

In [14]:
# initialize PTA
model = [s(psr) for psr in psrs]
pta = signal_base.PTA(model)

# set fixed white noise parameters
pta.set_default_params(setpars)

INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_efac to 1.16587
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_efac to 1.11896
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_efac to 1.08538
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_PUPPI_efac to 1.38104
INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_log10_equad to -7.93502
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_log10_equad to -6.17415
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_log10_equad to -6.51038
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_PUPPI_log10_equad to -6.53715
INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_log10_ecorr to -8.47348
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_log10_ecorr to -6.31096
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_log10_ecorr to -6.09208
INFO: enterprise.signals.signal_base: Setting B185

In [15]:
#Create starting locations for walkers
xecc = np.hstack(np.array([p.sample() for p in pta.params]))
ndim = len(xecc)

print(xecc)

[-1.0193696   2.65793668  1.77826861  4.25716882  0.44949878  4.81102683
  0.01083655  1.53016362  0.53220353  2.35841754 -0.69539408  5.71797668
  1.8815636   0.32469714  0.91462761  0.07929691 -1.22592147  3.56225791
  0.56030864  4.47163898  0.21106195  9.74444593  0.26592872]


In [16]:
#testing to see if we get a likelihood value
pta.get_lnlikelihood(xecc)

-8119296.057545082

In [17]:
# initialize pulsar distance parameters
p_dist_params = [ p for p in pta.param_names if 'p_dist' in p ]
for pd in p_dist_params:
    xecc[pta.param_names.index(pd)] = 0

In [18]:
# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)

In [19]:
snames = np.unique([[qq.signal_name for qq in pp._signals] 
                        for pp in pta._signalcollections])

In [20]:
groups = fns.get_parameter_groups(pta)

In [21]:
groups

[range(0, 23), [20, 21, 22], [], []]

In [22]:
groups = groups[:-2]

In [23]:
chaindir = '/home/bcheeseboro/nanograv_proj/enterprise_proj/pta_tutorial/ecc_chains/test/run3/'
resume = False
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, groups=groups, 
                 outDir=chaindir, resume=resume)

# write parameter file and parameter groups file
np.savetxt(chaindir + 'params.txt', list(map(str, pta.param_names)), fmt='%s')
np.savetxt(chaindir + 'groups.txt', groups, fmt='%s')

In [24]:
# add prior draws to proposal cycle
inc_psr_term = True
if inc_psr_term:
    psr_dist = {}
    for psr in psrs:
        psr_dist[psr.name] = psr.pdist
else:
    psr_dist = None
    
jp = fns.JumpProposal(pta, fgw=F0, psr_dist=psr_dist, 
                      rnposteriors=None, 
                      juporbposteriors=None)

In [25]:
sampler.addProposalToCycle(jp.draw_from_cw_prior, 20)
sampler.addAuxilaryJump(jp.fix_cyclic_pars)

In [26]:
filename = chaindir + '/params.txt'
np.savetxt(filename,list(map(str, pta.param_names)), fmt='%s')
np.savetxt(chaindir + 'groups.txt', groups, fmt='%s')

In [27]:
N = int(1.5e6)

In [28]:
sampler.sample(xecc, N, SCAMweight=30, AMweight=15, DEweight=50)

/home/bcheeseboro/anaconda3/envs/enterprise/lib/python3.6/site-packages/enterprise/signals/parameter.py:64: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))


Finished 0.67 percent in 1094.748648 s Acceptance rate = 0.76514Adding DE jump with weight 50
Finished 29.60 percent in 286873.369907 s Acceptance rate = 0.480115

KeyboardInterrupt: 